**Sections:**<a name="contents"></a>

[Control data download](#control_data_download)

[Create symbolink link copies for all experimental fastq files](#symb_link_input)

[Quality control](#QC_analysis)

In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

import warnings
warnings.simplefilter('ignore')

# general purpose packages
import pandas as pd
import numpy as np
import os
import json
import time
import re
import csv
import subprocess
import sys

import scipy.stats as stats
import statsmodels.stats as smstats
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import umap
import rpy2

from multiprocessing import Process, Manager, Pool
import multiprocessing
from functools import partial

from collections import Counter

import seaborn as sns; sns.set()

import matplotlib
matplotlib.style.use('seaborn')
matplotlib.use('Agg')
import matplotlib.pyplot as plt
matplotlib.rcParams['backend'] = "Qt5Agg"
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter

from IPython.display import display, Image

from adjustText import adjust_text
import builtins
%matplotlib inline

# for normalization
from sklearn.linear_model import QuantileRegressor

# for working with yaml files
import ruamel.yaml

# for working with genomic intervals
import pyranges as pr

import itertools

In [2]:
def get_pvalue_star(pval, thr=0.05):
    if thr == 0.05:
        if pval < 0.001:
            return "***"
        elif pval < 0.01:
            return "**"
        elif pval < 0.05:
            return "*"
        else:
            return "ns"
    elif thr == 0.1:
        if pval < 0.001:
            return "***"
        elif pval < 0.01:
            return "**"
        elif pval < 0.1:
            return "*"
        else:
            return "ns"

In [3]:
# paths to subdirectories
subdirs = {}

subdirs['main_project_dir'] = '/scicore/home/zavolan/GROUP/StefanieCLIP/'
subdirs['wf_dir'] = '/scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/'

subdirs['zarp_dir'] = '/scicore/home/zavolan/mirono0000/libs/zarp/'
subdirs['htsinfer_dir'] = '/scicore/home/zavolan/mirono0000/libs/htsinfer/'
subdirs['zarp_config_dir'] = subdirs['zarp_dir']+'config/'

subdirs['mouse_annotation_dir'] = '/scicore/home/zavolan/GROUP/Genomes/mus_musculus/'
subdirs['human_annotation_dir'] = '/scicore/home/zavolan/GROUP/Genomes/homo_sapiens/'

# shared project folder 
subdirs['shared_project_dir'] = subdirs['main_project_dir']+'aleksei/'
subdirs['temp_dir'] = subdirs['shared_project_dir']+'temp_dir/'
subdirs['slurm_dir'] = subdirs['temp_dir']+'slurm/'
subdirs['scripts_dir'] = subdirs['shared_project_dir']+'scripts/'
subdirs['figures_dir'] = subdirs['shared_project_dir']+'figures/'

subdirs['fastq_dir'] = subdirs['shared_project_dir']+'input_fastq/'
subdirs['metadata_dir'] = subdirs['shared_project_dir']+'metadata/'
subdirs['wf_output_dir'] = subdirs['shared_project_dir']+'output/'

# paths to files
file_paths = {}
### genome annotation files
file_paths['mouse_genome_file'] = subdirs['mouse_annotation_dir']+'GRCm39.primary_assembly.genome.fa'
file_paths['mouse_annotation_file'] = subdirs['mouse_annotation_dir']+'gencode.vM32.annotation.gtf'
file_paths['mouse_RNAcentral_annotation_file'] = subdirs['mouse_annotation_dir']+'mus_musculus.GRCm39.gff3.gz'

file_paths['mouse_prot_coding_gtf'] = subdirs['mouse_annotation_dir']+'coding.gencode.vM32.annotation.gtf'
file_paths['mouse_collapsed_prot_coding_gtf'] = subdirs['mouse_annotation_dir']+'collapsed.coding.gencode.vM32.annotation.gtf'

file_paths['mouse_enriched_annotation_file'] = subdirs['mouse_annotation_dir']+'enriched.gencode.vM32.annotation.gtf' # added RNA species from RNA central
file_paths['mouse_collapsed_enriched_annotation_file'] = subdirs['mouse_annotation_dir']+'collapsed.enriched.gencode.vM32.annotation.gtf'

file_paths['human_genome_file'] = subdirs['human_annotation_dir']+'GRCh38.primary_assembly.genome.fa'
file_paths['human_annotation_file'] = subdirs['human_annotation_dir']+'hg38_v42/gencode.v42.annotation.gtf'
file_paths['human_RNAcentral_annotation_file'] = subdirs['human_annotation_dir']+'hg38_v42/homo_sapiens.GRCh38.gff3.gz'

file_paths['human_enriched_annotation_file'] = subdirs['human_annotation_dir']+'hg38_v42/enriched.gencode.v42.annotation.gtf'

file_paths['htsinfer_transcripts_file'] = '/scicore/home/zavolan/GROUP/Genomes/htsinfer_deduplicated_transcripts.fasta'

### control data
file_paths['control_metadata_file_1'] = subdirs['metadata_dir']+'SRA_CONTROL_DATA_1.tsv'
file_paths['control_metadata_file_2'] = subdirs['metadata_dir']+'SRA_CONTROL_DATA_2.tsv'

os.system('mkdir -p '+' '.join(list(subdirs.values()))) # create all subdirs

0

# move bam files to dedicated folder

In [6]:
os.system("""find """+subdirs['wf_output_dir']+'samples/'+""" -name '*.0_um.sorted.bam' > """+subdirs['temp_dir']+"""low_dupl_files.tsv""")

0

In [8]:
os.system("""find """+subdirs['wf_output_dir']+'samples/'+""" -name '*.dedup.sorted.indexed.low_duplicate.bam' > """+subdirs['temp_dir']+"""low_dupl_files_1.tsv""")

0

In [24]:
low_dupl_files = pd.read_csv(subdirs['temp_dir']+"""low_dupl_files.tsv""",delimiter="\t",
                                   index_col=None,header=None)
low_dupl_files_1 = pd.read_csv(subdirs['temp_dir']+"""low_dupl_files_1.tsv""",delimiter="\t",
                                   index_col=None,header=None)
low_dupl_files['prior'] = 1
low_dupl_files_1['prior'] = 2
low_dupl_files = pd.concat([low_dupl_files,low_dupl_files_1]).reset_index(drop=True)
low_dupl_files['sample'] = low_dupl_files.apply(lambda x:x[0].split('/samples/')[1].split('/')[0],1)
low_dupl_files = low_dupl_files.sort_values(['sample','prior']).drop_duplicates('sample').reset_index(drop=True)
low_dupl_files[[0]].to_csv(subdirs['temp_dir']+"""low_dupl_files_to_retain.tsv""", sep=str('\t'),header=False,index=None,quoting=csv.QUOTE_NONE)

In [34]:
low_dupl_files[1] = low_dupl_files[0]+'.bai'

In [35]:
low_dupl_files[[1]].to_csv(subdirs['temp_dir']+"""low_dupl_bai_files_to_retain.tsv""", sep=str('\t'),header=False,index=None,quoting=csv.QUOTE_NONE)

In [11]:
low_dupl_files.iloc[0][0]

'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/N4BP2C_replicate1/map_genome_all/read_categories/0_um/N4BP2C_replicate1.0_um.sorted.bam'

In [12]:
low_dupl_files_1.iloc[0][0]

'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/N4BP2C_replicate1/map_genome_all/N4BP2C_replicate1.dedup.sorted.indexed.low_duplicate.bam'

In [7]:
subdirs['temp_dir']+"""low_dupl_files.tsv"""

'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/temp_dir/low_dupl_files.tsv'

In [32]:
"mkdir -p ./low_dupl_bam_files && cat "+subdirs['temp_dir']+"""low_dupl_files_to_retain.tsv"""+" | xargs -I % cp % "+'./low_dupl_bam_files'

'mkdir -p ./low_dupl_bam_files && cat /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/temp_dir/low_dupl_files_to_retain.tsv | xargs -I % cp % ./low_dupl_bam_files'

In [38]:
"mkdir -p ./low_dupl_bam_files && cat "+subdirs['temp_dir']+"""low_dupl_bai_files_to_retain.tsv"""+" | xargs -I % cp % "+'./low_dupl_bam_files'

'mkdir -p ./low_dupl_bam_files && cat /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/temp_dir/low_dupl_bai_files_to_retain.tsv | xargs -I % cp % ./low_dupl_bam_files'

In [40]:
"""tar -czvf low_duplication_bam_files.tar.gz ./low_dupl_bam_files"""

'tar -czvf low_duplication_bam_files.tar.gz ./low_dupl_bam_files'

# Make enriched gtf file for mouse and human

In [4]:
# organisms = ['mouse','human']
organisms = ['mouse']

for organism in organisms:
    command = 'samtools faidx '+file_paths[organism+'_genome_file']
    out = subprocess.check_output(command, shell=True)

    gtf_df = pd.read_csv(file_paths[organism+'_annotation_file'],delimiter="\t",index_col=None,header=None,skiprows=5)

    rna_central = pd.read_csv(file_paths[organism+'_RNAcentral_annotation_file'],delimiter="\t",index_col=None,header=None,skiprows=1,compression='gzip')
    rna_central[2] = rna_central[2].str.replace('noncoding_exon','exon')
    rna_central['gene_biotype'] = rna_central[8].str.split(';type=|;',expand=True)[1]
    rna_central['gene_source'] = 'RNA_central'
    rna_central['gene_id'] = rna_central[8].str.split('ID=|;|:',expand=True)[4]

    rna_central_exons = rna_central.loc[rna_central[2]=='exon'].copy().reset_index(drop=True)
    rna_central_exons['exon_id'] = rna_central_exons['gene_id']+'.transcript'+'.'+rna_central_exons[8].str.split('ID=|;|:',expand=True)[5]
    rna_central_exons['exon_number'] = rna_central_exons[8].str.split('ID=|;|:',expand=True)[5].str.split('exon',expand=True).iloc[:, -1]
    rna_central_exons[8] = 'gene_id "'+rna_central_exons['gene_id']+'"; transcript_id "'+rna_central_exons['gene_id']+'.transcript'+'"; exon_number "'+rna_central_exons['exon_number']+'"; gene_source "'+rna_central_exons['gene_source']+'"; gene_type "'+rna_central_exons['gene_biotype']+'"; transcript_source "'+rna_central_exons['gene_source']+'"; transcript_type "'+rna_central_exons['gene_biotype']+'"; exon_id "'+rna_central_exons['exon_id']+'"; tag "'+rna_central['gene_source']+'";'
    rna_central_exons['order']=3

    rna_central_exons['transcript_id'] = rna_central_exons['gene_id']+'.transcript'
    rna_central_exons['exon_coords'] = rna_central_exons[3].astype('str')+'_'+rna_central_exons[4].astype('str')+','
    rna_central_gr_transcripts = rna_central_exons.groupby([0,6,'transcript_id']).agg({'exon_coords':sum}).reset_index()
    rna_central_gr_transcripts['transcript_alt_id'] = rna_central_gr_transcripts[0].astype('str')+'_'+rna_central_gr_transcripts[6]+'_'+rna_central_gr_transcripts['exon_coords']

    gtf_df_exons = gtf_df.loc[gtf_df[2]=='exon'].reset_index(drop=True)
    gtf_df_exons['transcript_id'] = gtf_df_exons[8].str.split('transcript_id "',expand=True)[1].str.split('"',expand=True)[0]
    gtf_df_exons['exon_coords'] = gtf_df_exons[3].astype('str')+'_'+gtf_df_exons[4].astype('str')+','
    gtf_df_gr_transcripts = gtf_df_exons.groupby([0,6,'transcript_id']).agg({'exon_coords':sum}).reset_index()
    gtf_df_gr_transcripts['transcript_alt_id'] = gtf_df_gr_transcripts[0].astype('str')+'_'+gtf_df_gr_transcripts[6]+'_'+gtf_df_gr_transcripts['exon_coords']
    ensembl_transcripts = list(gtf_df_gr_transcripts['transcript_alt_id'].unique())

    preserve_transcripts_list = list(rna_central_gr_transcripts.loc[~rna_central_gr_transcripts['transcript_alt_id'].isin(ensembl_transcripts)]['transcript_id'].unique()) # when a transcript is present in ensemble and RNA central, prioritize ensemble
    rna_central_exons = rna_central_exons.loc[rna_central_exons['transcript_id'].isin(preserve_transcripts_list)].reset_index(drop=True)
    rna_central['transcript_id'] = rna_central['gene_id']+'.transcript'
    rna_central = rna_central.loc[rna_central['transcript_id'].isin(preserve_transcripts_list)].reset_index(drop=True)

    rna_central_exons = rna_central_exons[list(range(0,9))+['gene_id','order']]

    rna_central_transcripts = rna_central.loc[rna_central[2]=='transcript'].copy().reset_index(drop=True)
    rna_central_transcripts[8] = 'gene_id "'+rna_central_transcripts['gene_id']+'"; transcript_id "'+rna_central_transcripts['gene_id']+'.transcript'+'"; gene_source "'+rna_central_transcripts['gene_source']+'"; gene_type "'+rna_central_transcripts['gene_biotype']+'"; transcript_source "'+rna_central_transcripts['gene_source']+'"; transcript_type "'+rna_central_transcripts['gene_biotype']+'"; tag "'+rna_central_transcripts['gene_source']+'";'
    rna_central_transcripts['order']=2
    rna_central_transcripts = rna_central_transcripts[list(range(0,9))+['gene_id','order']]

    rna_central_genes = rna_central.loc[rna_central[2]=='transcript'].copy().reset_index(drop=True)
    rna_central_genes[2] = 'gene'
    rna_central_genes[8] = 'gene_id "'+rna_central_genes['gene_id']+'"; gene_source "'+rna_central_genes['gene_source']+'"; gene_type "'+rna_central_genes['gene_biotype']+'";'
    rna_central_genes['order']=1
    rna_central_genes = rna_central_genes[list(range(0,9))+['gene_id','order']]

    rna_central_gtf = pd.concat([rna_central_genes,rna_central_transcripts,rna_central_exons]).sort_values(['gene_id','order']).reset_index(drop=True).drop(['gene_id','order'],1)
    rna_central_gtf[0] = 'chr'+rna_central_gtf[0].astype('str') # so that the naming is consistent
    # save standard annotation enriched with RNA central
    enriched_gtf = pd.concat([gtf_df,rna_central_gtf]).reset_index(drop=True)
    genome_fai = pd.read_csv(file_paths[organism+'_genome_file']+'.fai',delimiter="\t",index_col=None,header=None)
    enriched_gtf = pd.merge(genome_fai[[0]],enriched_gtf,how='inner',on=0)

    # remove non-canonical chromosomes, they are not of interest
    enriched_gtf = enriched_gtf.loc[enriched_gtf[0].str.startswith('chr')].reset_index(drop=True)
    
    # check that inferred introns are consistent (start<end), otherwise remove such transcripts
    exons = enriched_gtf.loc[enriched_gtf[2]=='exon'].reset_index(drop=True)
    exons['transcript_id'] = exons[8].str.split('transcript_id "',expand=True)[1].str.split('";',expand=True)[0]
    exons = exons[[0,3,4,'transcript_id',6]]
    exons = exons.sort_values([0,6,'transcript_id',3,4],ascending=True).reset_index(drop=True)
    exons = exons.values
    introns = []
    intron_start, intron_end, prev_transcript = None, None, ''
    
    for exon in exons:
        if (intron_start is None) or (prev_transcript!=exon[3]):
            intron_start = exon[2]
            prev_transcript = exon[3]
        else:
            intron_end = exon[1]
            introns.append([exon[0],intron_start,intron_end,exon[3],0,exon[4]])
            intron_start = exon[2]
            prev_transcript = exon[3]
    introns = pd.DataFrame(introns)
    introns[2] = introns[2]-1
    introns = introns.sort_values([0,1,2],ascending=True).reset_index(drop=True)
    
    bad_transcripts = list(introns.loc[introns[1]>introns[2]][3].unique())
    
    transcripts = enriched_gtf.loc[enriched_gtf[2]=='transcript'].reset_index(drop=True)
    transcripts['transcript_id'] = transcripts[8].str.split('transcript_id "',expand=True)[1].str.split('";',expand=True)[0]
    transcripts['gene_id'] = transcripts[8].str.split('gene_id "',expand=True)[1].str.split('";',expand=True)[0]
    transcripts['bad'] = (transcripts['transcript_id'].isin(bad_transcripts)).astype('int')
    transcripts['t']=1
    gr = transcripts.groupby(['gene_id']).agg({'t':sum,'bad':sum}).reset_index()
    
    bad_genes = list(gr.loc[(gr['t']==gr['bad'])&(gr['bad']>0)]['gene_id'].unique())
    
    if len(bad_genes)>0:
        enriched_gtf = enriched_gtf.loc[~enriched_gtf[8].str.contains('|'.join(bad_genes))].reset_index(drop=True)
    if len(bad_transcripts):
        enriched_gtf = enriched_gtf.loc[~enriched_gtf[8].str.contains('|'.join(bad_transcripts))].reset_index(drop=True)
    
    enriched_gtf.to_csv(file_paths[organism+'_enriched_annotation_file'], sep=str('\t'),header=False,index=None,quoting=csv.QUOTE_NONE)

# Download control data<a name="control_data_download"></a>

In [18]:
samples_to_download_1 = pd.read_csv(subdirs['metadata_dir']+'SRA_CONTROL_DATA_1.tsv',delimiter="\t",
                                   index_col=None,header=0)
samples_to_download_2 = pd.read_csv(subdirs['metadata_dir']+'SRA_CONTROL_DATA_2.tsv',delimiter="\t",
                                   index_col=None,header=0)
samples_to_download_3 = pd.read_csv(subdirs['metadata_dir']+'SRA_CONTROL_DATA_3.tsv',delimiter="\t",
                                   index_col=None,header=0)
samples_to_download = pd.concat([samples_to_download_1,samples_to_download_2,samples_to_download_3]).reset_index(drop=True)

In [21]:
samples_to_download['expected_path'] = subdirs['fastq_dir']+samples_to_download['sample']+'/'+samples_to_download['sample']

In [27]:
big_command = ''
i=0
for index,row in samples_to_download.iterrows():
    SRA_id = row['sample']
    if not (os.path.isfile(row['expected_path']+'.fastq.gz') or os.path.isfile(row['expected_path']+'_1.fastq.gz')):
        command = 'prefetch -v -O '+subdirs['fastq_dir']+SRA_id+' '+SRA_id+\
        ' && fasterq-dump -f -v -e '+str(4)+' -O '+subdirs['fastq_dir']+SRA_id+' '+subdirs['fastq_dir']+SRA_id+'/'+SRA_id
        if i>0:
            big_command = big_command+' && '+command
        else:
            big_command = command
        i=i+1
big_command

'prefetch -v -O /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735700 SRR18735700 && fasterq-dump -f -v -e 4 -O /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735700 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735700/SRR18735700 && prefetch -v -O /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735702 SRR18735702 && fasterq-dump -f -v -e 4 -O /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735702 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735702/SRR18735702 && prefetch -v -O /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735704 SRR18735704 && fasterq-dump -f -v -e 4 -O /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735704 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735704/SRR18735704 && prefetch -v -O /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735706 SRR18735706 && fasterq-dump -f -v -e 4

In [35]:
# gzip fastq files
os.system("""find """+subdirs['fastq_dir']+""" -name '*.fastq' > """+subdirs['temp_dir']+"""fastq_file_paths.tsv""")
fastq_file_paths = pd.read_csv(subdirs['temp_dir']+"""fastq_file_paths.tsv""",delimiter="\t",
                                   index_col=None,header=None)
big_command = ''
i=0
for file_path in fastq_file_paths[0]:
    command = 'pigz -f --fast -p '+str(12)+' '+file_path
    if i>0:
        big_command = big_command+' && '+command
    else:
        big_command = command    
    i=i+1
big_command

'pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735702/SRR18735702_2.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735702/SRR18735702_1.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735704/SRR18735704_2.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735704/SRR18735704_1.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735698/SRR18735698_2.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735698/SRR18735698_1.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735706/SRR18735706_1.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735706/SRR18735706_2.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/a

In [36]:
# delete unnecessary .sra files
os.system("""find """+subdirs['fastq_dir']+""" -name '*.sra' > """+subdirs['temp_dir']+"""sra_file_paths.tsv""")
sra_file_paths = pd.read_csv(subdirs['temp_dir']+"""sra_file_paths.tsv""",delimiter="\t",
                                   index_col=None,header=None)
sra_file_paths.columns = ['path']
command = 'rm '+' '.join(sra_file_paths['path'])
# out = subprocess.check_output(command, shell=True)

In [ ]:
# make downloaded fastq.gz files read-only

In [ ]:
# chmod 554 -R ./input_fastq

# Make start_samples file

## for bCLIP and ChipSeq samples

In [39]:
# description file for bCLIP experimental samples
experimental_samples = pd.read_csv(subdirs['metadata_dir']+'experimental_bCLIP_samples.tsv',delimiter="\t",
                                   index_col=None,header=0)
exp_file_paths = experimental_samples[['lane_file','barcode_file']].drop_duplicates().reset_index(drop=True)

os.system("""find """+subdirs['main_project_dir']+'input/'+""" -name '*.fastq.gz' > """+subdirs['temp_dir']+"""exper_fastq_file_paths.tsv""")
fastq_file_paths = pd.read_csv(subdirs['temp_dir']+'exper_fastq_file_paths.tsv',delimiter="\t",
                                   index_col=None,header=None)
fastq_file_paths.columns = ['lane_file_path']
fastq_file_paths['lane_file'] = fastq_file_paths['lane_file_path'].str.split('/',expand=True).iloc[:, -1]
exp_file_paths = pd.merge(fastq_file_paths,exp_file_paths,how='inner',on=['lane_file'])

os.system("""find """+subdirs['main_project_dir']+'input/'+""" -name '*.fasta' > """+subdirs['temp_dir']+"""barcode_fasta_files.tsv""")
barcode_fasta_files = pd.read_csv(subdirs['temp_dir']+'barcode_fasta_files.tsv',delimiter="\t",
                                   index_col=None,header=None)
barcode_fasta_files.columns = ['barcode_file_path']
barcode_fasta_files['barcode_file'] = barcode_fasta_files['barcode_file_path'].str.split('/',expand=True).iloc[:, -1]
exp_file_paths = pd.merge(barcode_fasta_files,exp_file_paths,how='inner',on=['barcode_file'])

exp_file_paths['wfstart_lane_file_path'] = subdirs['fastq_dir']+exp_file_paths['lane_file'].str.replace('.fastq.gz','')+'/'+exp_file_paths['lane_file']
exp_file_paths['wfstart_barcode_file_path'] = subdirs['fastq_dir']+exp_file_paths['lane_file'].str.replace('.fastq.gz','')+'/'+exp_file_paths['barcode_file']

# for index, row in exp_file_paths.iterrows():
#     sample_name = row['lane_file'].replace('.fastq.gz','')
#     command = 'mkdir -p '+subdirs['fastq_dir']+sample_name+'/'
#     out = subprocess.check_output(command, shell=True)
#     command = 'rm -f '+row['wfstart_lane_file_path']+' && ln -f -s '+row['lane_file_path']+' '+row['wfstart_lane_file_path']
#     out = subprocess.check_output(command, shell=True)
#     command = 'rm -f '+row['wfstart_barcode_file_path']+' && ln -f -s '+row['barcode_file_path']+' '+row['wfstart_barcode_file_path']
#     out = subprocess.check_output(command, shell=True)

experimental_samples = pd.merge(experimental_samples,
                                exp_file_paths[['lane_file','barcode_file','wfstart_lane_file_path','wfstart_barcode_file_path']],
                                how='inner',on=['lane_file','barcode_file'])

experimental_samples = experimental_samples.drop(['lane_file','barcode_file'],1).rename(columns={'wfstart_lane_file_path':'lane_file','wfstart_barcode_file_path':'barcode_file'})

experimental_samples['genome_file'] = file_paths['mouse_genome_file']
experimental_samples['gtf_file'] = file_paths['mouse_annotation_file']
experimental_samples['lane_file_2'] = ''
experimental_samples = experimental_samples[['name','lane_name','lane_file','lane_file_2','kmer','barcode_file','exp_ctl','batch','condition_name','genome_file','gtf_file']]

experimental_samples['lane_name'] = 'f'+experimental_samples['barcode_file'].str.split('barcodes_',expand=True).iloc[:, -1].str.replace('.fasta','')
experimental_samples['organism'] = 'mouse'
experimental_samples['directionality'] = 's'
experimental_samples['library_layout'] = 'SINGLE'

# description file for public bCLIP samples
public_samples = pd.read_csv(subdirs['metadata_dir']+'public_bCLIP_and_ChipSeq_samples.tsv',delimiter="\t",
                                   index_col=None,header=0)
public_samples['genome_file'] = public_samples.apply(lambda x:file_paths['mouse_genome_file'] if x['batch'].endswith('_mESC') else file_paths['human_genome_file'],1)
public_samples['gtf_file'] = public_samples.apply(lambda x:file_paths['mouse_annotation_file'] if x['batch'].endswith('_mESC') else file_paths['human_annotation_file'],1)
public_samples['lane_name'] = 'f'+public_samples['name']
public_samples['organism'] = public_samples.apply(lambda x: 'mouse' if x['batch'].endswith('_mESC') else 'human',1)
public_samples['directionality'] = 's' # forward orientation of the first read
public_samples['library_layout'] = 'SINGLE'

# merge with public data
start_samples = pd.concat([public_samples,experimental_samples]).reset_index(drop=True)
start_samples['kmer'] = start_samples['kmer'].fillna('XXXXXXX')

# add list of chromosomes
organisms = list(start_samples['organism'].unique())
cl = []
for organism in organisms:
    enriched_gtf = pd.read_csv(file_paths[organism+'_enriched_annotation_file'],delimiter="\t",
                                       index_col=None,header=None,usecols = [0])
    chromosome_list =  list(enriched_gtf[0].unique())
    cl.append([organism,' '.join(chromosome_list)])
cl = pd.DataFrame(cl,columns=['organism','chromosomes'])
start_samples = pd.merge(start_samples,cl,how='left',on='organism')

start_samples.to_csv(subdirs['metadata_dir']+'start_bCLIP_and_ChipSeq_samples.tsv', sep=str('\t'),header=True,index=None,quoting=csv.QUOTE_NONE)

In [41]:
subdirs['metadata_dir']+'start_bCLIP_samples.tsv'

'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/metadata/start_bCLIP_samples.tsv'

In [47]:
subdirs['wf_output_dir']

'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/'

In [48]:
subdirs['slurm_dir']

'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/temp_dir/slurm/'

## for PRO-seq

In [40]:
# description file for public PRO-seq samples
public_samples = pd.read_csv(subdirs['metadata_dir']+'public_PROseq_samples.tsv',delimiter="\t",
                                   index_col=None,header=0)

public_samples['genome_file'] = public_samples.apply(lambda x:file_paths['mouse_genome_file'] if x['batch'].endswith('_mESC') else file_paths['human_genome_file'],1)
public_samples['gtf_file'] = public_samples.apply(lambda x:file_paths['mouse_annotation_file'] if x['batch'].endswith('_mESC') else file_paths['human_annotation_file'],1)
public_samples['lane_name'] = 'f'+public_samples['name']
public_samples['organism'] = public_samples.apply(lambda x: 'mouse' if x['batch'].endswith('_mESC') else 'human',1)
public_samples['directionality'] = 'S' # reverse orientation of the first read
public_samples['library_layout'] = 'SINGLE'

start_samples = public_samples.copy()
start_samples['kmer'] = start_samples['kmer'].fillna('XXXXXXX')

# add list of chromosomes
organisms = list(start_samples['organism'].unique())
cl = []
for organism in organisms:
    enriched_gtf = pd.read_csv(file_paths[organism+'_enriched_annotation_file'],delimiter="\t",
                                       index_col=None,header=None,usecols = [0])
    chromosome_list =  list(enriched_gtf[0].unique())
    cl.append([organism,' '.join(chromosome_list)])
cl = pd.DataFrame(cl,columns=['organism','chromosomes'])
start_samples = pd.merge(start_samples,cl,how='left',on='organism')
start_samples.to_csv(subdirs['metadata_dir']+'start_PROseq_samples.tsv', sep=str('\t'),header=True,index=None,quoting=csv.QUOTE_NONE)

## for RNA-seq

In [31]:
os.system("""find """+subdirs['main_project_dir']+'input/X204SC24014174-Z01-F001/01.RawData/'+""" -name '*.fq.gz' > """+subdirs['temp_dir']+"""exper_RNAseq_fastq_file_paths.tsv""")

0

In [39]:
exper_RNAseq_fastq_file_paths = pd.read_csv(subdirs['temp_dir']+"""exper_RNAseq_fastq_file_paths.tsv""",delimiter="\t",index_col=None,header=None)
exper_RNAseq_fastq_file_paths.columns = ['path']
exper_RNAseq_fastq_file_paths['name'] = exper_RNAseq_fastq_file_paths.apply(lambda x:x['path'].split('/')[-1][:-8],1)
exper_RNAseq_fastq_file_paths['lane_file'] = exper_RNAseq_fastq_file_paths.apply(lambda x:'/'.join(x['path'].split('/')[:-1])+'/'+x['name']+'_1.fq.gz',1)
exper_RNAseq_fastq_file_paths['lane_file_2'] = exper_RNAseq_fastq_file_paths.apply(lambda x:'/'.join(x['path'].split('/')[:-1])+'/'+x['name']+'_2.fq.gz',1)
experimental_samples = exper_RNAseq_fastq_file_paths.drop_duplicates('name').reset_index(drop=True)
experimental_samples['replicate'] = experimental_samples.apply(lambda x:x['name'].split('_')[0],1)

mouse_replicates = ['A12','A13','A14','A15','A18','A19']
human_replicates = list('A'+pd.Series(range(1,10)).astype('str'))

experimental_samples = experimental_samples.loc[experimental_samples['replicate'].isin(mouse_replicates+human_replicates)].reset_index(drop=True).drop('path',1)

experimental_samples_mouse = experimental_samples.loc[experimental_samples['replicate'].isin(mouse_replicates)].reset_index(drop=True)
experimental_samples_human = experimental_samples.loc[experimental_samples['replicate'].isin(human_replicates)].reset_index(drop=True)
experimental_samples_mouse['genome_file'] = file_paths['mouse_genome_file']
experimental_samples_human['genome_file'] = file_paths['human_genome_file']
experimental_samples_mouse['gtf_file'] = file_paths['mouse_annotation_file']
experimental_samples_human['gtf_file'] = file_paths['human_annotation_file']
experimental_samples_human['organism'] = 'human'
experimental_samples_mouse['organism'] = 'mouse'

# experimental_samples = pd.concat([experimental_samples_human,experimental_samples_mouse]).reset_index(drop=True)
experimental_samples = experimental_samples_human.copy()

experimental_samples = experimental_samples.drop('replicate',1)
experimental_samples['condition_name'] = 'NA' # define afterwards in analysis
experimental_samples['batch'] = 'new24_03'
experimental_samples['exp_ctl'] = 'NA'

# public_samples = pd.read_csv(subdirs['metadata_dir']+'public_RNAseq_samples.tsv',delimiter="\t",
#                                    index_col=None,header=0)
# public_samples['genome_file'] = file_paths['mouse_genome_file']
# public_samples['gtf_file'] = file_paths['mouse_annotation_file']
# public_samples['organism'] = 'mouse'

# start_samples = pd.concat([public_samples[list(experimental_samples.columns)],experimental_samples]).reset_index(drop=True)
start_samples = experimental_samples.copy()
start_samples['kmer'] = 'XXXXXXX'


# add list of chromosomes
organisms = list(start_samples['organism'].unique())
cl = []
for organism in organisms:
    enriched_gtf = pd.read_csv(file_paths[organism+'_enriched_annotation_file'],delimiter="\t",
                                       index_col=None,header=None,usecols = [0])
    chromosome_list =  list(enriched_gtf[0].unique())
    cl.append([organism,' '.join(chromosome_list)])
cl = pd.DataFrame(cl,columns=['organism','chromosomes'])
start_samples = pd.merge(start_samples,cl,how='left',on='organism')

start_samples.to_csv(subdirs['metadata_dir']+'start_RNAseq_samples.tsv', sep=str('\t'),header=True,index=None,quoting=csv.QUOTE_NONE)

In [38]:
len(start_samples)

35

# Run main WF

## bCLIP and ChipSeq analysis

In [17]:
command = """snakemake \
--snakefile """+subdirs['wf_dir']+"""Snakefile_bCLIP \
--configfile """+subdirs['wf_dir']+"""config_bCLIP.yaml \
--scheduler greedy \
--printshellcmds \
--use-conda --conda-frontend conda \
--use-singularity \
--singularity-args "--bind /scicore/home/zavolan/mirono0000/Projects/bCLIP/,/scicore/home/zavolan/GROUP/StefanieCLIP/,/scicore/home/zavolan/GROUP/Genomes/" \
--cluster-config """+subdirs['wf_dir']+"""cluster_bCLIP.json \
--cores 500 \
--local-cores 10 \
--jobs 100 \
--latency-wait 60 \
--cluster "sbatch \
--cpus-per-task={cluster.threads} \
--mem={cluster.mem} \
--qos={cluster.queue} \
--partition={cluster.partition} \
--time={cluster.time} \
--output={cluster.out}" \
--nolock \
-np"""
command

'snakemake --snakefile /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/Snakefile_bCLIP --configfile /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/config_bCLIP.yaml --scheduler greedy --printshellcmds --use-conda --conda-frontend conda --use-singularity --singularity-args "--bind /scicore/home/zavolan/mirono0000/Projects/bCLIP/,/scicore/home/zavolan/GROUP/StefanieCLIP/,/scicore/home/zavolan/GROUP/Genomes/" --cluster-config /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/cluster_bCLIP.json --cores 500 --local-cores 10 --jobs 100 --latency-wait 60 --cluster "sbatch --cpus-per-task={cluster.threads} --mem={cluster.mem} --qos={cluster.queue} --partition={cluster.partition} --time={cluster.time} --output={cluster.out}" --nolock -np'

## PRO-seq

In [14]:
command = """snakemake \
--snakefile """+subdirs['wf_dir']+"""Snakefile_PROseq \
--configfile """+subdirs['wf_dir']+"""config_PROseq.yaml \
--printshellcmds \
--use-conda --conda-frontend conda \
--use-singularity \
--singularity-args "--bind /scicore/home/zavolan/mirono0000/Projects/bCLIP/,/scicore/home/zavolan/GROUP/StefanieCLIP/,/scicore/home/zavolan/GROUP/Genomes/" \
--cluster-config """+subdirs['wf_dir']+"""cluster_PROseq.json \
--cores 500 \
--local-cores 10 \
--jobs 100 \
--latency-wait 60 \
--cluster "sbatch \
--cpus-per-task={cluster.threads} \
--mem={cluster.mem} \
--qos={cluster.queue} \
--partition={cluster.partition} \
--time={cluster.time} \
--output={cluster.out}" \
--nolock \
-np"""
command

'snakemake --snakefile /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/Snakefile_PROseq --configfile /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/config_PROseq.yaml --printshellcmds --use-conda --conda-frontend conda --use-singularity --singularity-args "--bind /scicore/home/zavolan/mirono0000/Projects/bCLIP/,/scicore/home/zavolan/GROUP/StefanieCLIP/,/scicore/home/zavolan/GROUP/Genomes/" --cluster-config /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/cluster_PROseq.json --cores 500 --local-cores 10 --jobs 100 --latency-wait 60 --cluster "sbatch --cpus-per-task={cluster.threads} --mem={cluster.mem} --qos={cluster.queue} --partition={cluster.partition} --time={cluster.time} --output={cluster.out}" --nolock -np'

## RNA-seq

In [30]:
command = """snakemake \
--snakefile """+subdirs['wf_dir']+"""Snakefile_RNAseq \
--configfile """+subdirs['wf_dir']+"""config_RNAseq.yaml \
--printshellcmds \
--use-conda --conda-frontend conda \
--use-singularity \
--singularity-args "--bind /scicore/home/zavolan/mirono0000/Projects/bCLIP/,/scicore/home/zavolan/GROUP/StefanieCLIP/,/scicore/home/zavolan/GROUP/Genomes/" \
--cluster-config """+subdirs['wf_dir']+"""cluster_RNAseq.json \
--cores 500 \
--local-cores 10 \
--jobs 100 \
--latency-wait 60 \
--cluster "sbatch \
--cpus-per-task={cluster.threads} \
--mem={cluster.mem} \
--qos={cluster.queue} \
--partition={cluster.partition} \
--time={cluster.time} \
--output={cluster.out}" \
--nolock \
-np"""
command

'snakemake --snakefile /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/Snakefile_RNAseq --configfile /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/config_RNAseq.yaml --printshellcmds --use-conda --conda-frontend conda --use-singularity --singularity-args "--bind /scicore/home/zavolan/mirono0000/Projects/bCLIP/,/scicore/home/zavolan/GROUP/StefanieCLIP/,/scicore/home/zavolan/GROUP/Genomes/" --cluster-config /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/cluster_RNAseq.json --cores 500 --local-cores 10 --jobs 100 --latency-wait 60 --cluster "sbatch --cpus-per-task={cluster.threads} --mem={cluster.mem} --qos={cluster.queue} --partition={cluster.partition} --time={cluster.time} --output={cluster.out}" --nolock -np'

## Auxiliary

In [ ]:
# run if want to remove all bam files
sample_file_name = subdirs['temp_dir']+'bam_file_paths.tsv'

os.system("""find """+'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/'+""" -name '*.Aligned.sortedByCoord.out.bam' > """+sample_file_name)
bam_file_paths = pd.read_csv(sample_file_name,delimiter="\t",
                                   index_col=None,header=None)
bam_file_paths['dir'] = bam_file_paths.apply(lambda x:os.path.dirname(x[0]), 1)
command = 'rm -r '+' '.join(bam_file_paths['dir'])
# out = subprocess.check_output(command, shell=True)

In [26]:
input_file_path = '/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/SRR18735676/segment_counts_all/binned_genome/SRR18735676.chr12.wd_counts.tsv'
sample = 'SRR18735676'
chr = 'chr12'
count_col = 'uniquely_mapped;0'
selected_TSSs_over_chrs_file = '/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/transcriptome/mouse/selected_TSSs_over_chrs_for_metaplots/'+chr+'.bed'
s_param = '-s'

metaplot_dir = '/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/SRR18735676/segment_counts_all/metaplot/'
bin_counts = pd.read_csv(input_file_path,delimiter="\t",index_col=None,header=0,dtype={"chr": "category","strand":"category","segment_name":"category"},nrows=5)

In [27]:
i=1
col_number = -1
for elem in bin_counts.columns:
    if elem==count_col:
        col_number = i
    i=i+1
if col_number==-1:
    score_val = "0"
else:
    score_val = "$"+str(col_number)

nonsorted_temp_file_path =metaplot_dir+sample+'.'+chr+'.nonsorted.bed'
sorted_temp_file_path = metaplot_dir+sample+'.'+chr+'.sorted.bed'
sel_file_path = metaplot_dir+sample+'.'+chr+'.selected.bed'

command = """awk \'(NR>1) {print $1"\\t"$2"\\t"$3"\\t""t""\\t\""""+score_val+"""\"\\t"$4}' """+input_file_path+' > '+nonsorted_temp_file_path
out = subprocess.check_output(command, shell=True)

command = 'bedtools sort -i '+nonsorted_temp_file_path+' > '+sorted_temp_file_path
out = subprocess.check_output(command, shell=True)   
command = 'bedtools intersect -sorted -a '+sorted_temp_file_path+' -b '+selected_TSSs_over_chrs_file+' -wa -wb -s'+""" | awk \'{print $1"\\t"$2"\\t"$3"\\t"$10"\\t"$5"\\t"$6}\' > """+sel_file_path
out = subprocess.check_output(command, shell=True)

command = 'rm '+sorted_temp_file_path+' '+nonsorted_temp_file_path
out = subprocess.check_output(command, shell=True)

In [28]:
bin_counts = pd.read_csv(sel_file_path,delimiter="\t",index_col=None,header=None)
bin_counts.columns = ['chr','start','end','name','r','strand']
bin_counts['TSS'] = bin_counts['name'].astype('str').str.split(';',expand=True)[1].astype('int')

bin_counts['pos_rel_to_TSS'] = ((bin_counts['end']+bin_counts['start'])*0.5-bin_counts['TSS']).astype('int')*((bin_counts['strand']=='+').astype('int')*2-1)
bin_counts['pos_rel_to_TSS'] = (np.round((bin_counts['pos_rel_to_TSS']/10),0)*10).astype('int')

In [32]:
bin_counts.loc[bin_counts['name'].str.contains('Fos')].head(150).tail(50)

,chr,start,end,name,r,strand,TSS,pos_rel_to_TSS
78005,chr12,85520651,85520661,control_up;85520664;37534;Fos,1.116,+,85520664,-10
78006,chr12,85520661,85520663,control_up;85520664;37534;Fos,5.465,+,85520664,0
78007,chr12,85520664,85520674,signal;85520664;37534;Fos,21.436,+,85520664,0
78008,chr12,85520674,85520684,signal;85520664;37534;Fos,35.726,+,85520664,20
78009,chr12,85520684,85520694,signal;85520664;37534;Fos,35.542,+,85520664,20
78010,chr12,85520694,85520704,signal;85520664;37534;Fos,27.344,+,85520664,40
78011,chr12,85520704,85520714,signal;85520664;37534;Fos,17.330,+,85520664,40
78012,chr12,85520714,85520724,signal;85520664;37534;Fos,9.099,+,85520664,60
78013,chr12,85520724,85520734,signal;85520664;37534;Fos,3.184,+,85520664,60
78014,chr12,85520734,85520744,signal;85520664;37534;Fos,1.229,+,85520664,80


In [13]:
bin_counts = bin_counts.drop_duplicates(['chr','start','end','strand']).reset_index(drop=True)


,chr,start,end,strand,segment_name,uniquely_mapped;0,uniquely_mapped;1,multimapped;0,multimapped;1
0,chr13,3057720,3057730,+,five_prime_only_gene_flank;38636,0.0,0.0,0.0,0.0
1,chr13,3057730,3057740,+,five_prime_only_gene_flank;38636,0.0,0.0,0.0,0.0
2,chr13,3057740,3057750,+,five_prime_only_gene_flank;38636,0.0,0.0,0.0,0.0
3,chr13,3057750,3057760,+,five_prime_only_gene_flank;38636,0.0,0.0,0.0,0.0
4,chr13,3057760,3057770,+,five_prime_only_gene_flank;38636,0.0,0.0,0.0,0.0


In [67]:
#  TT-seq
expr_ctl, expr_KD = 37, 26
library_ctl,library_KD = 94, 91
np.log2((expr_KD/library_KD)/(expr_ctl/library_ctl))

-0.46221943600891674

In [61]:
#  lab RNA-seq
expr_ctl, expr_KD = 10, 12
library_ctl,library_KD = 70, 57
np.log2((expr_KD/library_KD)/(expr_ctl/library_ctl))

0.5594274086140185

In [66]:
#  RNA-seq SI paper
expr_ctl, expr_KD = 24,28
library_ctl,library_KD = 58, 57
np.log2((expr_KD/library_KD)/(expr_ctl/library_ctl))

0.24748340229927837

In [74]:
#  RNA-seq degron paper
expr_ctl, expr_KD = 14,17
library_ctl,library_KD = 72, 55
np.log2((expr_KD/library_KD)/(expr_ctl/library_ctl))

0.6686732071103879

In [27]:
k1,k2 = 15,6
stats.fisher_exact([[k1,10**5],[k2,10**5]])[1],stats.fisher_exact([[k1,10**3],[k2,10**3]])[1],stats.binomtest(k=k1,n=k1+k2,alternative='two-sided').pvalue

(0.07833836473519383, 0.07687943978759948, 0.0783538818359375)

In [ ]:
s=$(zcat /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/INTS11C_replicate10_exp5/map_genome/read_categories/2_um/INTS11C_replicate10_exp5.2_um.fastq.gz | head -n 2 | tail -n 1 | awk '{print length}')
l=$((s<150 ? s : 150))

In [4]:
"""picard FilterSamReads I=/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/SRR15070640/map_genome/SRR15070640.dedup.sorted.indexed.bam \
O=/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/SRR15070640/map_genome/read_categories/5_um/picard_test.sam \
READ_LIST_FILE=/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/SRR15070640/map_genome/read_categories/5_um/SRR15070640.5_um.read_names.txt \
FILTER=includeReadList"""

'picard FilterSamReads I=/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/SRR15070640/map_genome/SRR15070640.dedup.sorted.indexed.bam O=/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/SRR15070640/map_genome/read_categories/5_um/picard_test.sam READ_LIST_FILE=/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/SRR15070640/map_genome/read_categories/5_um/SRR15070640.5_um.read_names.txt FILTER=includeReadList'

In [7]:
command = """python /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/scripts/get_genome_segmentations.py \
--input_gtf /scicore/home/zavolan/GROUP/Genomes/mus_musculus/enriched.gencode.vM32.annotation.gtf \
--input_genome_fai /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/genome_indices/mouse/genome.fai \
--output_exon_intron_gs /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/transcriptome/mouse/exon_intron_genome_segmentation.bed \
--output_binned_gs /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/transcriptome/mouse/binned_genome_segmentation.bed \
--bin_size 10 \
--temp_dir /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/transcriptome/mouse/temp \
--output_modified_gtf /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/transcriptome/mouse/modified_annotation.gtf \
--gene_flank 1000"""
command

'python /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/scripts/get_genome_segmentations.py --input_gtf /scicore/home/zavolan/GROUP/Genomes/mus_musculus/enriched.gencode.vM32.annotation.gtf --input_genome_fai /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/genome_indices/mouse/genome.fai --output_exon_intron_gs /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/transcriptome/mouse/exon_intron_genome_segmentation.bed --output_binned_gs /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/transcriptome/mouse/binned_genome_segmentation.bed --bin_size 10 --temp_dir /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/transcriptome/mouse/temp --output_modified_gtf /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/transcriptome/mouse/modified_annotation.gtf --gene_flank 1000'

In [ ]:
bedtools bamtobed -split -cigar -i /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/NRDE2_replicate3_exp10/map_genome/NRDE2_replicate3_exp10.Aligned.sortedByCoord.out.bam

In [ ]:
set +o pipefail; bedtools bamtobed -cigar -i {input.bam} | bedtools groupby -g 4 -c 7 -o distinct | awk '$2 !~ /,/' | awk '$2 !~ /N/' | bedtools groupby -g 2 -c 2 -o count > {output.cigar_frequencies}

In [22]:
import itertools

grouped_bed_file_path = '/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/RBC_replicate1/map_genome/RBC_replicate1.dedup.sorted.indexed.grouped.bed'
bed_file_path = '/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/RBC_replicate1/map_genome/RBC_replicate1.dedup.sorted.indexed.bed'

outdir = os.path.dirname(grouped_bed_file_path)+'/read_categories/'
sample_id = os.path.basename(grouped_bed_file_path).replace('.dedup.sorted.indexed.grouped.bed','')

d_cats = [0,1,2,3,4,5]
mm_modes = ['um','mm']

read_categories = list(itertools.product(*[d_cats,mm_modes])) # cartesian product

grouped_bed_file = pd.read_csv(grouped_bed_file_path,delimiter="\t",index_col=None,header=None)
grouped_bed_file.columns = [0,1,2,'name','w',5,'d','wd','d_cat']

for read_category in read_categories:
    cat_name = '_'.join(list(pd.Series(read_category).astype('str')))
    out_subdir = outdir+cat_name+'/'
    command = 'mkdir -p '+out_subdir
    out = subprocess.check_output(command, shell=True)    

    output_read_names_path = out_subdir+sample_id+'.'+cat_name+'.read_names.txt'
    output_read_weights_path = out_subdir+sample_id+'.'+cat_name+'.read_weights.txt'
    
    d_cat = read_category[0]
    if read_category[1]=='um':
        cur_reads = bed_file.loc[(bed_file['d_cat']==d_cat)&(bed_file['w']==1)]
    else:
        cur_reads = bed_file.loc[(bed_file['d_cat']==d_cat)&(bed_file['w']<1)]
    if len(cur_reads)==0:
        command = 'echo "empty" > '+output_read_names_path
        out = subprocess.check_output(command, shell=True)
        command = 'echo "0" > '+output_read_weights_path
        out = subprocess.check_output(command, shell=True)
        continue
    cur_reads = cur_reads.sort_values([0,1,2])reset_index(drop=True)
    cur_reads[[0,1,2,'name','w',5]].to_csv(out_subdir+'temp1.bed', sep=str('\t'),header=True,index=None)
    
    command = 'bedtools intersect -a '+out_subdir+'temp1.bed -b '+bed_file_path+' -wo -f 1.0 -r -s | bedtools groupby -g <QNAME> -c <w> -o first > '+out_subdir+'temp2.bed'
    out = subprocess.check_output(command, shell=True)
    
    command = 'cut -f1 '+out_subdir+'temp2.bed > '+output_read_names_path
    out = subprocess.check_output(command, shell=True)
    command = 'cut -f2 '+out_subdir+'temp2.bed > '+output_read_weights_path
    out = subprocess.check_output(command, shell=True)

    command = 'rm '+out_subdir+'temp2.bed '+out_subdir+'temp1.bed'
    out = subprocess.check_output(command, shell=True)

'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/RBC_replicate1/map_genome/read_categories/'

In [23]:
read_categories

[(0, 'um'),
 (0, 'mm'),
 (1, 'um'),
 (1, 'mm'),
 (2, 'um'),
 (2, 'mm'),
 (3, 'um'),
 (3, 'mm'),
 (4, 'um'),
 (4, 'mm'),
 (5, 'um'),
 (5, 'mm')]

In [18]:
# building bracken dbs for all lengths present
kraken_db = '/scicore/home/zavolan/GROUP/Genomes/KRAKEN_DB/standard/'
start_samples = pd.read_csv(subdirs['metadata_dir']+'start_samples.tsv',delimiter="\t",index_col=None,header=0)

read_lengths_file_name = subdirs['temp_dir']+'read_lengths.tsv'
read_lengths = pd.read_csv(subdirs['wf_output_dir']+'misc/max_read_lengths.tsv',delimiter="\t",index_col=None,header=0)
read_lengths = read_lengths[['max_read_length_estimate']].drop_duplicates().rename(columns = {'max_read_length_estimate':'index_size'}).reset_index(drop=True)

read_lengths['id'] = read_lengths.index+1
read_lengths[['id','index_size']].to_csv(read_lengths_file_name, sep=str('\t'),header=False,index=None)

max_node_mem = 512
mem = 400
cpus = min(15,int(128/((max_node_mem*0.8)/mem)))
print(str(cpus))

f = open(subdirs['scripts_dir']+'bracken_build.sbatch', "w")

preambula = \
"""#!/bin/bash
  
#SBATCH --job-name=bracken_build
#SBATCH --time=5:00:00
#SBATCH --qos=6hours
#SBATCH --output="""+subdirs['slurm_dir']+"""%A_%a.out
#SBATCH --error="""+subdirs['slurm_dir']+"""%A_%a.err
#SBATCH --cpus-per-task="""+str(cpus)+"""
#SBATCH --mem="""+str(mem)+"""G
#SBATCH --array=1-"""+str(len(read_lengths))+"""%200

source ~/.bashrc
"""
f.write(preambula)
f.write("""
read_length=$(awk -v i=$SLURM_ARRAY_TASK_ID '{ if ($1 == i) { print $2} }' """+read_lengths_file_name+""")
echo $read_length
""")

command = """bracken-build -d """+kraken_db+""" -t """+str(cpus)+""" -k 35 -l $read_length"""
f.write(command)
f.close()

print('sbatch '+subdirs['scripts_dir']+'bracken_build.sbatch')

15
sbatch /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/scripts/bracken_build.sbatch


In [4]:
# do kraken and bracken for unmapped reads
kraken_db = '/scicore/home/zavolan/GROUP/Genomes/KRAKEN_DB/standard/'

start_samples = pd.read_csv(subdirs['metadata_dir']+'start_samples.tsv',delimiter="\t",index_col=None,header=0)
# get length estimates for samples
l = []
for sample in list(start_samples['name']):
    rl = pd.read_csv(subdirs['wf_output_dir']+'samples/'+sample+'/read_length/'+sample+'.max_read_length.txt',header=None).loc[0,0]
    l.append(rl)
start_samples['index_size'] = l
start_samples = start_samples.rename(columns={'name':'sample'})

sample_file_name = subdirs['temp_dir']+'unmapped_fastq_file_paths.tsv'

os.system("""find """+subdirs['wf_output_dir']+""" -name '*.Unmapped.out.fastq.gz' > """+sample_file_name)
fastq_file_paths = pd.read_csv(sample_file_name,delimiter="\t",
                                   index_col=None,header=None)
fastq_file_paths.columns = ['path']
fastq_file_paths['output_dir'] = fastq_file_paths['path'].str.split('map_genome/',expand=True)[0]+'kraken/'
fastq_file_paths['sample'] = fastq_file_paths['path'].str.split('map_genome/',expand=True)[1].str.replace('.Unmapped.out.fastq.gz','')
fastq_file_paths = pd.merge(fastq_file_paths,start_samples[['sample','index_size']].rename(columns={'index_size':'expected_read_length'}),how='inner',on='sample')
fastq_file_paths['kraken_output_file'] = fastq_file_paths['output_dir']+fastq_file_paths['sample']+'.unmapped.kraken'
fastq_file_paths['kreport_output_file'] = fastq_file_paths['output_dir']+fastq_file_paths['sample']+'.unmapped.kreport'
fastq_file_paths['bracken_output_file'] = fastq_file_paths['output_dir']+fastq_file_paths['sample']+'.unmapped.bracken'
fastq_file_paths['id'] = fastq_file_paths.index+1
fastq_file_paths[['id','path','output_dir','kraken_output_file','kreport_output_file','bracken_output_file','expected_read_length']].to_csv(sample_file_name, sep=str('\t'),header=False,index=None)

max_node_mem = 512
mem = 100
cpus = int(128/((max_node_mem*0.8)/mem))
print(str(cpus))

f = open(subdirs['scripts_dir']+'kraken_unmapped_reads.sbatch', "w")

preambula = \
"""#!/bin/bash
  
#SBATCH --job-name=kraken_unmapped_reads
#SBATCH --time=1:00:00
#SBATCH --qos=6hours
#SBATCH --output="""+subdirs['slurm_dir']+"""%A_%a.out
#SBATCH --error="""+subdirs['slurm_dir']+"""%A_%a.err
#SBATCH --cpus-per-task="""+str(cpus)+"""
#SBATCH --mem="""+str(mem)+"""G
#SBATCH --array=1-"""+str(len(fastq_file_paths))+"""%200

source ~/.bashrc
"""
f.write(preambula)
f.write("""
fastq_file_path=$(awk -v i=$SLURM_ARRAY_TASK_ID '{ if ($1 == i) { print $2} }' """+sample_file_name+""")
output_dir=$(awk -v i=$SLURM_ARRAY_TASK_ID '{ if ($1 == i) { print $3} }' """+sample_file_name+""")
kraken_output_file=$(awk -v i=$SLURM_ARRAY_TASK_ID '{ if ($1 == i) { print $4} }' """+sample_file_name+""")
kreport_output_file=$(awk -v i=$SLURM_ARRAY_TASK_ID '{ if ($1 == i) { print $5} }' """+sample_file_name+""")
bracken_output_file=$(awk -v i=$SLURM_ARRAY_TASK_ID '{ if ($1 == i) { print $6} }' """+sample_file_name+""")
expected_read_length=$(awk -v i=$SLURM_ARRAY_TASK_ID '{ if ($1 == i) { print $7} }' """+sample_file_name+""")
echo $fastq_file_path
echo $output_dir
""")

command = """mkdir -p $output_dir"""
f.write(command+'\n') 

command = """
kraken2 \
--threads """+str(cpus)+""" \
--db """+kraken_db+""" \
--report $kreport_output_file \
$fastq_file_path > \
$kraken_output_file"""
f.write(command)

command = """ && bracken \
-d """+kraken_db+""" \
-i $kreport_output_file \
-o $bracken_output_file \
-r $expected_read_length -l S -t 1
"""
f.write(command)

f.close()

print('sbatch '+subdirs['scripts_dir']+'kraken_unmapped_reads.sbatch')

31
sbatch /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/scripts/kraken_unmapped_reads.sbatch


In [5]:
# do kraken and bracken for mapped reads
kraken_db = '/scicore/home/zavolan/GROUP/Genomes/KRAKEN_DB/standard/'

start_samples = pd.read_csv(subdirs['metadata_dir']+'start_samples.tsv',delimiter="\t",index_col=None,header=0)
# get length estimates for samples
l = []
for sample in list(start_samples['name']):
    rl = pd.read_csv(subdirs['wf_output_dir']+'samples/'+sample+'/read_length/'+sample+'.max_read_length.txt',header=None).loc[0,0]
    l.append(rl)
start_samples['index_size'] = l
start_samples = start_samples.rename(columns={'name':'sample'})

sample_file_name = subdirs['temp_dir']+'mapped_fastq_file_paths.tsv'

os.system("""find """+subdirs['wf_output_dir']+""" -name '*.Aligned.sortedByCoord.out.fastq.gz' > """+sample_file_name)
fastq_file_paths = pd.read_csv(sample_file_name,delimiter="\t",
                                   index_col=None,header=None)
fastq_file_paths.columns = ['path']
fastq_file_paths['output_dir'] = fastq_file_paths['path'].str.split('map_genome/',expand=True)[0]+'kraken/'
fastq_file_paths['sample'] = fastq_file_paths['path'].str.split('map_genome/',expand=True)[1].str.replace('.Aligned.sortedByCoord.out.fastq.gz','')
fastq_file_paths = pd.merge(fastq_file_paths,start_samples[['sample','index_size']].rename(columns={'index_size':'expected_read_length'}),how='inner',on='sample')
fastq_file_paths['kraken_output_file'] = fastq_file_paths['output_dir']+fastq_file_paths['sample']+'.mapped.kraken'
fastq_file_paths['kreport_output_file'] = fastq_file_paths['output_dir']+fastq_file_paths['sample']+'.mapped.kreport'
fastq_file_paths['bracken_output_file'] = fastq_file_paths['output_dir']+fastq_file_paths['sample']+'.mapped.bracken'
fastq_file_paths['id'] = fastq_file_paths.index+1
fastq_file_paths[['id','path','output_dir','kraken_output_file','kreport_output_file','bracken_output_file','expected_read_length']].to_csv(sample_file_name, sep=str('\t'),header=False,index=None)

max_node_mem = 512
mem = 100
cpus = int(128/((max_node_mem*0.8)/mem))
print(str(cpus))

f = open(subdirs['scripts_dir']+'kraken_mapped_reads.sbatch', "w")

preambula = \
"""#!/bin/bash
  
#SBATCH --job-name=kraken_mapped_reads
#SBATCH --time=1:00:00
#SBATCH --qos=6hours
#SBATCH --output="""+subdirs['slurm_dir']+"""%A_%a.out
#SBATCH --error="""+subdirs['slurm_dir']+"""%A_%a.err
#SBATCH --cpus-per-task="""+str(cpus)+"""
#SBATCH --mem="""+str(mem)+"""G
#SBATCH --array=1-"""+str(len(fastq_file_paths))+"""%200

source ~/.bashrc
"""
f.write(preambula)
f.write("""
fastq_file_path=$(awk -v i=$SLURM_ARRAY_TASK_ID '{ if ($1 == i) { print $2} }' """+sample_file_name+""")
output_dir=$(awk -v i=$SLURM_ARRAY_TASK_ID '{ if ($1 == i) { print $3} }' """+sample_file_name+""")
kraken_output_file=$(awk -v i=$SLURM_ARRAY_TASK_ID '{ if ($1 == i) { print $4} }' """+sample_file_name+""")
kreport_output_file=$(awk -v i=$SLURM_ARRAY_TASK_ID '{ if ($1 == i) { print $5} }' """+sample_file_name+""")
bracken_output_file=$(awk -v i=$SLURM_ARRAY_TASK_ID '{ if ($1 == i) { print $6} }' """+sample_file_name+""")
expected_read_length=$(awk -v i=$SLURM_ARRAY_TASK_ID '{ if ($1 == i) { print $7} }' """+sample_file_name+""")
echo $fastq_file_path
echo $output_dir
""")

command = """mkdir -p $output_dir"""
f.write(command+'\n') 

command = """
kraken2 \
--threads """+str(cpus)+""" \
--db """+kraken_db+""" \
--report $kreport_output_file \
$fastq_file_path > \
$kraken_output_file"""
f.write(command)

command = """ && bracken \
-d """+kraken_db+""" \
-i $kreport_output_file \
-o $bracken_output_file \
-r $expected_read_length -l S -t 1
"""
f.write(command)

f.close()

print('sbatch '+subdirs['scripts_dir']+'kraken_mapped_reads.sbatch')

31
sbatch /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/scripts/kraken_mapped_reads.sbatch
